In [18]:
import types

class fucker:
    def __init__(self, cum_amount, fuck_cunt_cnt):
        self.cum_amount = cum_amount
        self.fuck_cunt_cnt = fuck_cunt_cnt

    def fuck(self, girl:str):
        print(f"Yes!!!{self.cum_amount}")
        self.fuck_cunt_cnt +=1

Qiyu = fucker(114, 514)

def new_fuck(self, girl:str):
    self.fuck_cunt_cnt +=1
    print(f"{girl}, I'm fucking cumming! you're the {self.fuck_cunt_cnt}-th girl I've fucked.\n\
          You'll recieve {self.cum_amount} mL of my cum.")

Qiyu.fuck = types.MethodType(new_fuck, Qiyu)

Qiyu.fuck("Girl")

Girl, I'm fucking cumming! you're the 515-th girl I've fucked.
          You'll recieve 114 mL of my cum.


In [19]:
import copy
class new_fucker(fucker):
    def __init__(self, cum_amount, fuck_cunt_cnt):
        super().__init__(cum_amount, fuck_cunt_cnt)
        self.old_fuck = copy.copy(self.fuck)
        def fuck(girl:str):
            print("I'm fucking horny!")
            self.old_fuck(girl)
        self.fuck = fuck

me = new_fucker(114, 514)

me.fuck("Girl")

I'm fucking horny!
Yes!!!114


In [21]:
hasattr(me, 'old_fuck')

True

In [13]:
import torch
import matplotlib.pyplot as plt
import sys
from neuralop.models import FNO

from neuralop import Trainer
from neuralop.training import OutputEncoderCallback
from neuralop.utils import count_params
from neuralop import LpLoss, H1Loss

device = 'cuda'

from neuralop.datasets.autoregressive_dataset import load_autoregressive_traintestsplit
data_path = "/home/yichen/repo/cfd/myFNO/data/zongyi/NavierStokes_V1e-5_N1200_T20.mat"

n_train = 1000
n_test = 200
batch_size = 32
test_batch_size = 64
train_subsample_rate = 1
test_subsample_rate = 1
time_step = 1
train_loader, test_loader = load_autoregressive_traintestsplit(
    data_path,
    n_train, n_test,
    batch_size, test_batch_size, 
    train_subsample_rate, test_subsample_rate,
    time_step,
    predict_feature='u',
)

Below we would first establish a similar dataset adapter.

input: train loader

output: a new test loader

transform rule:
u1 = u / k; 
visc1 = visc / k; 
f1 = f / k**2; 

index rule: 
if the k list has length $n_k$, then for idx

In [2]:
model_name = "TFNO_prodlayer"

In [3]:
# We create a tensorized FNO model

input_prods = None
n_modes=21

model = FNO(in_channels=3, n_modes=(n_modes, n_modes), hidden_channels=32, 
             projection_channels=64, factorization=None, channel_mixing='prod-layer', stabilizer='tanh', rank=0.)

# method_name = type(model).__name__.split('.')[-1]

model = model.to(device)

n_params = count_params(model)
print(f'\nOur '+model_name+f' model has {n_params} parameters.')

sys.stdout.flush()


Our TFNO_prodlayer model has 1658657 parameters.


In [4]:
#Create the optimizer
optimizer = torch.optim.Adam(model.parameters(), 
                                lr=1e-3, 
                                weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=100, gamma=0.5)

# Creating the losses
l2loss = LpLoss(d=2, p=2)
h1loss = H1Loss(d=2)

train_loss = h1loss
eval_losses={'h1': h1loss, 'l2': l2loss}

print('\n### MODEL ###\n', model)
print('\n### OPTIMIZER ###\n', optimizer)
print('\n### SCHEDULER ###\n', scheduler)
print('\n### LOSSES ###')
print(f'\n * Train: {train_loss}')
print(f'\n * Test: {eval_losses}')
sys.stdout.flush()


### MODEL ###
 FNO(
  (fno_blocks): FNOBlocks1(
    (convs): SpectralConv(
      (weight): ModuleList(
        (0-7): 8 x ComplexDenseTensor(shape=torch.Size([32, 32, 10, 10]), rank=None)
      )
    )
    (fno_skips): ModuleList(
      (0-3): 4 x Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
    )
    (channel_mixer): ModuleList(
      (0-3): 4 x ProductLayer(
        (linear): MLP(
          (fcs): ModuleList(
            (0): Conv2d(34, 32, kernel_size=(1, 1), stride=(1, 1))
          )
        )
      )
    )
    (mixer_skips): ModuleList(
      (0-3): 4 x SoftGating()
    )
  )
  (lifting): MLP(
    (fcs): ModuleList(
      (0): Conv2d(3, 256, kernel_size=(1, 1), stride=(1, 1))
      (1): Conv2d(256, 32, kernel_size=(1, 1), stride=(1, 1))
    )
  )
  (projection): MLP(
    (fcs): ModuleList(
      (0): Conv2d(32, 64, kernel_size=(1, 1), stride=(1, 1))
      (1): Conv2d(64, 1, kernel_size=(1, 1), stride=(1, 1))
    )
  )
)

### OPTIMIZER ###
 Adam (
Parameter Group

In [5]:
import time
localtime = time.localtime(time.time())
time_now = f"{localtime.tm_mon}-{localtime.tm_mday}-{localtime.tm_hour}-{localtime.tm_min}"

In [6]:
from neuralop.training import MultipleInputCallback, SimpleTensorBoardLoggerCallback
trainer = Trainer(model=model, n_epochs=500,
                  device=device,
                  callbacks=[MultipleInputCallback(append_positional_encoding=True), SimpleTensorBoardLoggerCallback(log_dir='runs/TorisLi_exp_'+model_name+time_now,)],             
                  wandb_log=False,
                  log_test_interval=4,
                  use_distributed=False,
                  verbose=True)

2024-01-19 15:26:07.688335: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-19 15:26:07.815762: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-19 15:26:07.815802: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-19 15:26:07.836270: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-19 15:26:07.881064: I tensorflow/core/platform/cpu_feature_guar

using standard method to load data to device.
using standard method to compute loss.
self.override_load_to_device=False
self.overrides_loss=False


In [7]:
trainer.train(train_loader=train_loader,
              test_loaders={64: test_loader},
              optimizer=optimizer, 
              scheduler=scheduler, 
              regularizer=False, 
              training_loss=train_loss, 
              eval_losses=eval_losses)

Training on 19000 samples
Testing on [3800] samples         on resolutions [64].
Raw outputs of size out.shape=torch.Size([32, 1, 64, 64])
[0] time=7.22, avg_loss=13.8714, train_err=11.6763, 64_h1=0.2767, 64_l2=0.1016
[4] time=6.28, avg_loss=7.4647, train_err=6.2834, 64_h1=0.1992, 64_l2=0.0669


KeyboardInterrupt: 

In [ ]:
import os
import shutil
import tempfile
import torch

temp_dir = tempfile.gettempdir()
temp_file_path = os.path.join(temp_dir, 'temp_model.pth')
torch.save(trainer.model.state_dict(), temp_file_path)

target_path = './test_param_search/b_64/mode_21/prod_2/layer_4/hid_32/lift_256/proj_64/fact-/rank_0.42/mix-/pos-enc-True/lr_0.001/wd_0/sche-step_100/gamma_0.5/loss-h1/TorusLi_FNO_1-17-19-20.pth'

target_dir = os.path.dirname(target_path)
if not os.path.exists(target_dir):
    os.makedirs(target_dir)

shutil.move(temp_file_path, target_path)

'./test_param_search/b_64/mode_21/prod_2/layer_4/hid_32/lift_256/proj_64/fact-/rank_0.42/mix-/pos-enc-True/lr_0.001/wd_0/sche-step_100/gamma_0.5/loss-h1/TorusLi_FNO_1-17-19-20.pth'

In [ ]:
import os
path = './test_param_search/b_64/mode_21/prod_2/TorusLi_FNO_1-17-19-20.pth'

# Create directories if they do not exist
dir_path = os.path.dirname(path)
if not os.path.exists(dir_path):
    os.makedirs(dir_path)

# Save the model
torch.save(trainer.model.state_dict(), path)

True


In [10]:
torch.save(trainer.model.state_dict(), "./ckpt/TorisLi/"+model_name+"_ep500.pth")

In [14]:
test_samples = test_loader.dataset

fig = plt.figure(figsize=(7, 7))
for index in range(3):
    data = test_samples[index]
    # Input x
    x = data['x']
    # Ground-truth
    y = data['y']
    # Model prediction
    out = model(x.unsqueeze(0))

    ax = fig.add_subplot(3, 3, index*3 + 1)
    ax.imshow(x[0], cmap='gray')
    if index == 0: 
        ax.set_title('Input x')
    plt.xticks([], [])
    plt.yticks([], [])

    ax = fig.add_subplot(3, 3, index*3 + 2)
    ax.imshow(y.squeeze())
    if index == 0: 
        ax.set_title('Ground-truth y')
    plt.xticks([], [])
    plt.yticks([], [])

    ax = fig.add_subplot(3, 3, index*3 + 3)
    ax.imshow(out.squeeze().detach().numpy())
    if index == 0: 
        ax.set_title('Model prediction')
    plt.xticks([], [])
    plt.yticks([], [])

fig.suptitle('Inputs, ground-truth output and prediction.', y=0.98)
plt.tight_layout()
fig.show()

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/yichen/anaconda3/envs/test/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_44388/849268122.py", line 11, in <module>
    out = model(x.unsqueeze(0))
  File "/home/yichen/anaconda3/envs/test/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1501, in _call_impl
    return forward_call(*args, **kwargs)
  File "/home/yichen/repo/cfd/myNeuralOperator/neuralop/models/new_fno.py", line 252, in forward
  File "/home/yichen/anaconda3/envs/test/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1501, in _call_impl
    return forward_call(*args, **kwargs)
  File "/home/yichen/repo/cfd/myNeuralOperator/neuralop/layers/mlp.py", line 62, in forward
    x = fc(x)
  File "/home/yichen/anaconda3/envs/test/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1501, in _call_impl
    return forward_call(*args

<Figure size 700x700 with 0 Axes>